# This is Cartography put together into one place. 

# Imports Section 

In [ ]:
import pandas as pd
import altair as alt
import numpy as np
from scipy.spatial.distance import squareform, pdist
import pandas as pd
import numpy as np
from Bio import SeqIO
import seaborn as sns
import re
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from augur.utils import json_to_tree
import json
from sklearn.manifold import MDS
from sklearn.manifold import TSNE
import umap
from scipy.stats import linregress
from pathlib import Path
import statsmodels
import statistics
import matplotlib.pyplot as plt
from Helpers import get_euclidean_data_frame, get_hamming_distances, linking_tree_with_plots_brush
from Helpers import linking_tree_with_plots_clickable
from Helpers import scatterplot_xyvalues, scatterplot_tooltips, scatterplot_with_tooltip_interactive

%matplotlib inline

## Pathogen-specific variables

Consider consolidating these into a single configuration file that can be passed to the notebook as a command line argument for more scriptable generation of these figures.

In [ ]:
path = "../zika-nextstrain/results/aligned.fasta"
dropped_strains = ["Yunnan/01/2019"]
distance_matrix_file = "DistanceMatrixZika.csv"
tree_path = '../zika-nextstrain/auspice/zika-cartography_tree.json'

### Reading in the Fasta File
- I used BioPython to parse the Fasta file into two numpy Arrays: Genomes and Strains. 

In [ ]:
#work on making this work
strains = []
genomes = []
for record in SeqIO.parse(path, "fasta"):
    if(record.id not in dropped_strains):
        strains.append(str(record.id))
        genomes.append(str(record.seq))

#### Checking to make sure the file I picked is a aligned Fasta file / is the file I wanted

In [ ]:
genomes_missing_bases = []
for x in genomes:
    x = re.sub(r'[^AGCT]', '5', x)
    numberOfN = x.count("5")
    genomes_missing_bases.append(numberOfN)

In [ ]:
genomes_missing_bases_df = pd.DataFrame(genomes_missing_bases)
genomes_missing_bases_df = genomes_missing_bases_df.merge(genomes_df, how='outer', left_index = True, right_index = True)
genomes_missing_bases_df.columns = ["bases missing", "strain"]

# Missing Bases

In [ ]:
genomes_missing_bases_df.head()

In [ ]:
alt.Chart(genomes_missing_bases_df).mark_bar().encode(
    alt.X("bases missing:Q", bin=True),
    y="count()"
)

In [ ]:
# Also, consider storing the list of dropped strains in a text file outside of the notebook.
dropped_strains.extend(list(genomes_missing_bases_df[genomes_missing_bases_df["bases missing"]>1000]["strain"]))

In [ ]:
#work on making this work
strains = []
genomes = []
for record in SeqIO.parse(path, "fasta"):
    if(record.id not in dropped_strains):
        strains.append(str(record.id))
        genomes.append(str(record.seq))

In [ ]:
len(strains)
print(len(genomes[0]) == len(genomes[1]))
print(len(genomes))

In [ ]:
strains[:5]

In [ ]:
strains_df = pd.DataFrame(genomes)
genomes_df = pd.DataFrame(strains)
genomes_df.columns = ["strain"]

# Creating the Distance Matrix
- I used Hamming Distance to find the pairwise distance between each genome and each other genome, effectively creating a similarity/distance matrix
    - In my Hamming Distance method, I only counted something as a different if it was a mismatch between the nucleotides (A,G,C, or T), not gaps (as that was throwing off the algorithm too much for smaller strains)
- I then used Seaborn to generate a heatmap to make sure the matrix looked correct

In [ ]:
%%time
# Try to load an existing distance matrix. Create it, if it doesn't already exist.
try:
    # The index should be the first column and correspond to strain name for the row.
    similarity_matrix = pd.read_csv(distance_matrix_file, index_col=0)
    print("Loaded existing distance matrix")
except FileNotFoundError:
    print("Could not find existing distance matrix, creating it now", end="...")
    
    # Calculate Hamming distances.
    hamming_distances = get_hamming_distances(genomes)
    
    # Convert distinct pairwise distances into the more redundant but more interpretable square matrix.
    similarity_matrix = squareform(hamming_distances)
    
    # Convert the numpy matrix to a pandas data frame with strain annotations for rows and columns.
    similarity_matrix = pd.DataFrame(
        similarity_matrix,
        columns=strains,
        index=strains
    )
    
    # Write out the resulting data frame to cache distance calculations.
    # Keep the index in the output file, so it is immediately available on read.
    similarity_matrix.to_csv(distance_matrix_file)
    print("done!")

In [ ]:
similarity_matrix

In [ ]:
sns.heatmap(similarity_matrix)

# Creating the Phylogenetic Tree in Altair
- I used Altair to make this tree (Documentation linked [here][1]
- I opened and imported the json from a build from NextStrain ([flu][2], [zika][3], etc)
- The data from the JSON and the Data from the tree are usually a little different, so after merging the two dataframes you may get some errors.

[1]: https://altair-viz.github.io/index.html
[2]: https://github.com/nextstrain/seasonal-flu
[3]: https://altair-viz.github.io/index.html

In [ ]:
with open(tree_path) as fh:
    json_tree_handle = json.load(fh)

In [ ]:
tree = json_to_tree(json_tree_handle)

In [ ]:
tree

In [ ]:
node_data = [
    {
        "strain": node.name,
        "date": node.attr["num_date"],
        "y": node.yvalue,
        "region": node.attr["region"],
        "country": node.attr["country"],
        "parent_date": node.parent is not None and node.parent.attr["num_date"] or node.attr["num_date"],
        "parent_y": node.parent is not None and node.parent.yvalue or node.yvalue,
        "clade_membership" : node.attr['clade_membership']
    }
    for node in tree.find_clades(terminal=True)
]

In [ ]:
node_data[10]

In [ ]:
node_df = pd.DataFrame(node_data)

In [ ]:
node_df.head()

In [ ]:
node_df["y"] = node_df["y"].max() - node_df["y"]

In [ ]:
node_df["parent_y"] = node_df["parent_y"].max() - node_df["parent_y"]

In [ ]:
node_df.shape

In [ ]:
node_df.head()

In [ ]:
node_df["region"].unique()

In [ ]:
# Reannotate clades that we aren't interested in as "other" to simplify color assignment in visualizations.
try:
    node_df["clade_membership_color"] = node_df["clade_membership"].apply(lambda clade: clade if clade in clades_to_plot else "other")
except:
    node_df["clade_membership_color"] = node_df["clade_membership"]
    print("clades_to_plot undefined")

In [ ]:
indices_to_drop = similarity_matrix[~similarity_matrix.index.isin(node_df["strain"])].dropna(how = 'all')
similarity_matrix = similarity_matrix[similarity_matrix.index.isin(node_df["strain"])].dropna(how = 'all')
similarity_matrix = similarity_matrix.drop(indices_to_drop.index, axis=1)
similarity_matrix

In [ ]:
node_df.head()

## Checking for Outliers in Pairwise Distance

In [ ]:
mean_distances = similarity_matrix.mean().reset_index(name="mean_distance").rename(columns={"index": "strain"})

In [ ]:
mean_distances.head()

In [ ]:
alt.Chart(mean_distances, height=150).mark_boxplot().encode(
    x = alt.X('mean_distance', title="mean of pairwise distances"),
    tooltip = ["strain"]
)

# Running PCA on Scaled and Centered Data
- I treated each nucleotide as a "site", or dimension, and found the probability of having a certain nucleotide given the frequency of that letter at that site.
- I used [this paper][1] as my source 
- The equation is as follows where C is the matrix of dimensions, M is the mean, and p is the frequency of a nucleotide at that given site. 
![](https://journals.plos.org/plosgenetics/article/file?type=thumbnail&id=info:doi/10.1371/journal.pgen.0020190.e003)

In [ ]:
numbers = genomes[:]
for i in range(0,len(genomes)):
    numbers[i] = re.sub(r'[^AGCT]', '5', numbers[i])
    numbers[i] = list(numbers[i].replace('A','1').replace('G','2').replace('C', '3').replace('T','4'))
    numbers[i] = [int(j) for j in numbers[i]]
genomes_df = pd.DataFrame(numbers)
genomes_df.columns = ["Site " + str(k) for k in range(0,len(numbers[i]))]

In [ ]:
genomes_df.head()

In [ ]:
#performing PCA on my pandas dataframe 
pca = PCA(n_components=10,svd_solver='full') #can specify n, since with no prior knowledge, I use None
principalComponents = pca.fit_transform(genomes_df)

In [ ]:
# Create a data frame from the PCA embedding.
principalDf = pd.DataFrame(data = principalComponents, columns = ["PCA" + str(i) for i in range(1,11)])

# Annotate rows by their original strain names. PCA rows are in the same order as
# the `genomes` rows which are in the same order as the `strains` rows.
principalDf["strain"] = strains

In [ ]:
df = pd.concat([pd.DataFrame(np.arange(1,11)), pd.DataFrame([round(pca.explained_variance_ratio_[i],4) for i in range(0,len(pca.explained_variance_ratio_))])], axis = 1)
df.columns = ['principal components','explained variance']
df

In [ ]:
alt.Chart(df).mark_point().encode(
    x='principal components:Q',
    y='explained variance:Q'
)

In [ ]:
merged_pca_df = principalDf.merge(node_df, on="strain")

In [ ]:
merged_pca_df.head()

In [ ]:
list_of_chart = linking_tree_with_plots_brush(merged_pca_df,['PCA1','PCA2','PCA3','PCA4'],
                                         ['PCA1 (Explained Variance : {}%'.format(round(pca.explained_variance_ratio_[0]*100,2)) + ")",
                                          'PCA2 (Explained Variance : {}%'.format(round(pca.explained_variance_ratio_[1]*100,2)) + ")",
                                          'PCA3 (Explained Variance : {}%'.format(round(pca.explained_variance_ratio_[2]*100,2)) + ")",
                                          'PCA4 (Explained Variance : {}%'.format(round(pca.explained_variance_ratio_[3]*100,2)) + ")"],
                                         "clade_membership:N",['strain','region'])
chart = list_of_chart[0]|list_of_chart[1]|list_of_chart[2]
chart.save("../Docs/PCAZikaBrush.html")
chart

In [ ]:
PCA_violin_df = get_euclidean_data_frame(merged_pca_df, "PCA1", "PCA2", "PCA")
g = sns.FacetGrid(
    PCA_violin_df,
    col="embedding",
    col_wrap=3,
    sharey=False,
    height=4
)
g = g.map(sns.violinplot, "clade_status", "distance", order=["within", "between"])
g.set_axis_labels("Clade status", "Distance")
plt.savefig("../docs/PCAViolinPlotZika.png")

In [ ]:
total_df = scatterplot_xyvalues(strains, similarity_matrix, merged_pca_df, "PCA1", "PCA2", "PCA")
y_values = statsmodels.nonparametric.smoothers_lowess.lowess(
    total_df["euclidean"],
    total_df["genetic"],
    frac=0.6666666666666666,
    it=3,
    delta=0.0,
    is_sorted=False,
    missing='drop',
    return_sorted=True
)

PD_Y_values = pd.DataFrame(y_values)
PD_Y_values.columns = ["LOWESS_x", "LOWESS_y"]

regression = linregress(total_df["genetic"], total_df["euclidean"])
slope, intercept, r_value, p_value, std_err = regression

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(total_df["genetic"], total_df["euclidean"], "o", alpha=0.25)
ax.plot(PD_Y_values["LOWESS_x"], PD_Y_values["LOWESS_y"], label="LOESS")

ax.set_xlabel("Genetic distance")
ax.set_ylabel("Euclidean distance (PCA)")
ax.set_title("PCA Euclidean distance vs. genetic distance ($R^2=%.3f$)" % (r_value ** 2))

sns.despine()
plt.savefig("../docs/PCAScatterplotLOESSZika.png")

# Running MDS on the Dataset

In [ ]:
embedding = MDS(n_components=2,metric=True,dissimilarity='precomputed')
X_transformed = embedding.fit_transform(similarity_matrix)

In [ ]:
raw_stress = embedding.stress_
normalized_stress = np.sqrt(raw_stress /((similarity_matrix.values.ravel() ** 2).sum() / 2))
print(normalized_stress.round(2))

In [ ]:
MDS_df = pd.DataFrame(X_transformed,columns=['MDS' + str(i) for i in range(1,3)])

In [ ]:
# Annotate rows by their original strain names. The same logic from PCA holds here
# and for later embeddings.
MDS_df["strain"] = similarity_matrix.index

In [ ]:
merged_mds_df = MDS_df.merge(node_df, on="strain")

In [ ]:
merged_mds_df.head()

In [ ]:
chart_12_mds = scatterplot_with_tooltip_interactive(merged_mds_df,'MDS1','MDS2',"MDS1","MDS2",['strain','clade_membership'],'clade_membership_color')
#chart_34_mds = scatterplot_with_tooltip_interactive(merged_mds_df,'MDS3','MDS4',"MDS3","MDS4",['strain','clade_membership'],'clade_membership_color')
#chart_56_mds = scatterplot_with_tooltip_interactive(merged_mds_df,'MDS5','MDS6',"MDS5","MDS6",['strain','clade_membership'],'clade_membership_color')
chart_12_mds#|chart_34_mds|chart_56_mds

In [ ]:
chart_MDS = scatterplot_tooltips(strains, similarity_matrix, merged_mds_df, "MDS1", "MDS2", "MDS", 4000)
#chart_34_scatter = scatterplot_tooltips(strains, similarity_matrix, merged_mds_df, "MDS3", "MDS4", "MDS", 4000)
#chart_56_scatter = scatterplot_tooltips(strains, similarity_matrix, merged_mds_df, "MDS5", "MDS6", "MDS", 4000)
chart_MDS #| chart_34_scatter | chart_56_scatter

In [ ]:
total_df = scatterplot_xyvalues(strains, similarity_matrix, merged_mds_df, "MDS1", "MDS2", "MDS")
y_values = statsmodels.nonparametric.smoothers_lowess.lowess(
    total_df["euclidean"],
    total_df["genetic"],
    frac=0.6666666666666666,
    it=3,
    delta=0.0,
    is_sorted=False,
    missing='drop',
    return_sorted=True
)

PD_Y_values = pd.DataFrame(y_values)
PD_Y_values.columns = ["LOWESS_x", "LOWESS_y"]

regression = linregress(total_df["genetic"], total_df["euclidean"])
slope, intercept, r_value, p_value, std_err = regression

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(total_df["genetic"], total_df["euclidean"], "o", alpha=0.25)
ax.plot(PD_Y_values["LOWESS_x"], PD_Y_values["LOWESS_y"], label="LOESS")

ax.set_xlabel("Genetic distance")
ax.set_ylabel("Euclidean distance (MDS)")
ax.set_title("MDS Euclidean distance vs. genetic distance ($R^2=%.3f$)" % (r_value ** 2))

sns.despine()
plt.savefig("../docs/MDSScatterplotLOESSZika.png")

In [ ]:
MDS_violin_df = get_euclidean_data_frame(merged_mds_df, "MDS1", "MDS2", "MDS")
g = sns.FacetGrid(
    MDS_violin_df,
    col="embedding",
    col_wrap=3,
    sharey=False,
    height=4
)
g = g.map(sns.violinplot, "clade_status", "distance", order=["within", "between"])
g.set_axis_labels("Clade status", "Distance")
plt.savefig("../docs/MDSViolinPlotZika.png")

In [ ]:
list_of_data_and_titles = ['MDS1','MDS2']#,'MDS3','MDS4','MDS5','MDS6']
list_of_chart = linking_tree_with_plots_brush(
    merged_mds_df,
    list_of_data_and_titles,
    list_of_data_and_titles,
    'clade_membership_color',
    ["clade_membership","strain:N"]
)
chart = list_of_chart[0]|list_of_chart[1]#|list_of_chart[2]|list_of_chart[3]
chart

In [ ]:
chart.save("../Docs/MDSZikaBrush.html")

# Running T-SNE on the Dataset 

In [ ]:
embedding = TSNE(n_components=2,metric='precomputed',perplexity = 25.95)
X_transformed = embedding.fit_transform(similarity_matrix)

In [ ]:
TSNE_df = pd.DataFrame(X_transformed,columns=['TSNE' + str(i) for i in range(1,3)])

In [ ]:
TSNE_df["strain"] = similarity_matrix.index

In [ ]:
TSNE_df.head()

In [ ]:
merged_tsne_df = TSNE_df.merge(node_df, on="strain")

In [ ]:
scatterplot_with_tooltip_interactive(merged_tsne_df,'TSNE1','TSNE2',"TSNE1","TSNE2",['strain','clade_membership'],'clade_membership_color')

In [ ]:
list_of_chart = linking_tree_with_plots_brush(
    merged_tsne_df,
    ['TSNE1','TSNE2'],
    ['TSNE1','TSNE2'],
    'clade_membership_color',
    ["clade_membership:N","strain:N"]
)
chart = list_of_chart[0]|list_of_chart[1]
chart
chart.save("../Docs/TSNEZikaBrush.html")

In [ ]:
TSNE_violin_df = get_euclidean_data_frame(merged_tsne_df, "TSNE1", "TSNE2", "TSNE")
g = sns.FacetGrid(
    TSNE_violin_df,
    col="embedding",
    col_wrap=3,
    sharey=False,
    height=4
)
g = g.map(sns.violinplot, "clade_status", "distance", order=["within", "between"])
g.set_axis_labels("Clade status", "Distance")
plt.savefig("../docs/TSNEViolinPlotZika.png")

In [ ]:
total_df = scatterplot_xyvalues(strains, similarity_matrix, merged_tsne_df, "TSNE1", "TSNE2", "TSNE")
y_values = statsmodels.nonparametric.smoothers_lowess.lowess(
    total_df["euclidean"],
    total_df["genetic"],
    frac=0.6666666666666666,
    it=3,
    delta=0.0,
    is_sorted=False,
    missing='drop',
    return_sorted=True
)

PD_Y_values = pd.DataFrame(y_values)
PD_Y_values.columns = ["LOWESS_x", "LOWESS_y"]

regression = linregress(total_df["genetic"], total_df["euclidean"])
slope, intercept, r_value, p_value, std_err = regression

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(total_df["genetic"], total_df["euclidean"], "o", alpha=0.25)
ax.plot(PD_Y_values["LOWESS_x"], PD_Y_values["LOWESS_y"], label="LOESS")

ax.set_xlabel("Genetic distance")
ax.set_ylabel("Euclidean distance (t-SNE)")
ax.set_title("t-SNE Euclidean distance vs. genetic distance ($R^2=%.3f$)" % (r_value ** 2))

sns.despine()
plt.savefig("../docs/TSNEScatterplotZika.png")

# Running UMAP on the Dataset

In [ ]:
reducer = umap.UMAP(n_neighbors=150,
        min_dist=.7,
        n_components=2,
        init="spectral")
embedding = reducer.fit_transform(similarity_matrix)

In [ ]:
UMAP_df = pd.DataFrame(embedding,columns=['UMAP' + str(i) for i in range(1,3)])

In [ ]:
UMAP_df["strain"] = similarity_matrix.index

In [ ]:
UMAP_df.head()

In [ ]:
merged_umap_df = UMAP_df.merge(node_df, on="strain")

In [ ]:
merged_umap_df.head()

In [ ]:
merged_umap_df.shape

In [ ]:
scatterplot_with_tooltip_interactive(merged_umap_df,'UMAP1','UMAP2',"UMAP1","UMAP2",['strain','clade_membership'],'clade_membership_color')

In [ ]:
list_of_data_and_titles = ['UMAP1','UMAP2']
list_of_chart = linking_tree_with_plots_brush(
    merged_umap_df,
    list_of_data_and_titles,
    list_of_data_and_titles,
    'clade_membership_color',
    ["clade_membership","strain:N"]
)
chart = list_of_chart[0]|list_of_chart[1]
chart

In [ ]:
chart.save("../Docs/UMAPZikaBrush.html")

In [ ]:
UMAP_violin_df = get_euclidean_data_frame(merged_umap_df, "UMAP1", "UMAP2", "UMAP")
g = sns.FacetGrid(
    UMAP_violin_df,
    col="embedding",
    col_wrap=3,
    sharey=False,
    height=4
)
g = g.map(sns.violinplot, "clade_status", "distance", order=["within", "between"])
g.set_axis_labels("Clade status", "Distance")
plt.savefig("../docs/UMAPViolinPlotZika.png")

In [ ]:
total_df = scatterplot_xyvalues(strains, similarity_matrix, merged_umap_df, "UMAP1", "UMAP2", "UMAP")
y_values = statsmodels.nonparametric.smoothers_lowess.lowess(
    total_df["euclidean"],
    total_df["genetic"],
    frac=0.6666666666666666,
    it=3,
    delta=0.0,
    is_sorted=False,
    missing='drop',
    return_sorted=True
)

PD_Y_values = pd.DataFrame(y_values)
PD_Y_values.columns = ["LOWESS_x", "LOWESS_y"]

regression = linregress(total_df["genetic"], total_df["euclidean"])
slope, intercept, r_value, p_value, std_err = regression

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(total_df["genetic"], total_df["euclidean"], "o", alpha=0.25)
ax.plot(PD_Y_values["LOWESS_x"], PD_Y_values["LOWESS_y"], label="LOESS")

ax.set_xlabel("Genetic distance")
ax.set_ylabel("Euclidean distance (UMAP)")
ax.set_title("UMAP Euclidean distance vs. genetic distance ($R^2=%.3f$)" % (r_value ** 2))

sns.despine()
plt.savefig("../docs/UMAPScatterplotZika.png")

# Linking all plots together clickable with Tree

In [ ]:
merged_df = node_df.merge(
    principalDf,
    on="strain"
).merge(
    MDS_df,
    on="strain"
).merge(
    TSNE_df,
    on="strain"
).merge(
    UMAP_df,
    on="strain"
)

In [ ]:
merged_df.shape

In [ ]:
merged_df.head()

In [ ]:
data = linking_tree_with_plots_clickable(
    merged_df,
    ['MDS1', 'MDS2','TSNE1', 'TSNE2', 'PCA1', 'PCA2', 'UMAP1', 'UMAP2'],
    ['MDS1', 'MDS2', 'TSNE1', 'TSNE2','PCA1 (Expected Variance : {}%'.format(round(pca.explained_variance_ratio_[0]*100,2)) + ")",
    'PCA2 (Expected Variance : {}%'.format(round(pca.explained_variance_ratio_[1]*100,2)) + ")",'UMAP1','UMAP2'],
    'clade_membership_color:N',
    ['clade_membership'],
    ['strain','clade_membership']
)

In [ ]:
data1 = linking_tree_with_plots_brush(
    merged_df,
    ['TSNE1', 'TSNE2','UMAP1', 'UMAP2'],
    ['TSNE1', 'TSNE2','UMAP1','UMAP2'],
    'clade_membership_color:N',
    ['strain','clade_membership']
)

In [ ]:
TSNEUMAP = data1[0]|data1[1]|data1[2]
TSNEUMAP.save('../Docs/TSNEUMAPClickableZika.html')
TSNEUMAP

In [ ]:
PCAMDS = data[3]|data[1]|data[5]
TSNEUMAP = data[2]|data[4]
embeddings = alt.vconcat(PCAMDS,TSNEUMAP)
embeddings
fullChart = alt.hconcat(data[0],embeddings)
fullChart.save('../Docs/FullLinkedChartClickableZika.html')
fullChart

## Scatterplots for all embeddings 
Concatenating all embedding data frames to plot genetic vs Euclidean distance for each embedding

In [ ]:
umap_df = scatterplot_xyvalues(strains, similarity_matrix, merged_umap_df, "UMAP1", "UMAP2", "UMAP")
tsne_df = scatterplot_xyvalues(strains, similarity_matrix, merged_tsne_df, "TSNE1", "TSNE2", "TSNE")
mds_df = scatterplot_xyvalues(strains, similarity_matrix, merged_mds_df, "MDS1", "MDS2", "MDS")
pca_df = scatterplot_xyvalues(strains, similarity_matrix, merged_pca_df, "PCA1", "PCA2", "PCA")

y_values_umap = statsmodels.nonparametric.smoothers_lowess.lowess(
    umap_df["euclidean"],
    umap_df["genetic"],
    frac=0.6666666666666666,
    it=3,
    delta=0.0,
    is_sorted=False,
    missing='drop',
    return_sorted=True
)

y_values_tsne = statsmodels.nonparametric.smoothers_lowess.lowess(
    tsne_df["euclidean"],
    tsne_df["genetic"],
    frac=0.6666666666666666,
    it=3,
    delta=0.0,
    is_sorted=False,
    missing='drop',
    return_sorted=True
)

y_values_mds = statsmodels.nonparametric.smoothers_lowess.lowess(
    mds_df["euclidean"],
    mds_df["genetic"],
    frac=0.6666666666666666,
    it=3,
    delta=0.0,
    is_sorted=False,
    missing='drop',
    return_sorted=True
)

y_values_pca = statsmodels.nonparametric.smoothers_lowess.lowess(
    pca_df["euclidean"],
    pca_df["genetic"],
    frac=0.6666666666666666,
    it=3,
    delta=0.0,
    is_sorted=False,
    missing='drop',
    return_sorted=True
)

PD_Y_values_umap = pd.DataFrame(y_values_umap)
PD_Y_values_umap.columns = ["LOWESS_x", "LOWESS_y"]

regression_umap = linregress(umap_df["genetic"], umap_df["euclidean"])
slope, intercept, r_value_umap, p_value, std_err = regression_umap

PD_Y_values_tsne = pd.DataFrame(y_values_tsne)
PD_Y_values_tsne.columns = ["LOWESS_x", "LOWESS_y"]

regression_tsne = linregress(tsne_df["genetic"], tsne_df["euclidean"])
slope, intercept, r_value_tsne, p_value, std_err = regression_tsne

PD_Y_values_mds = pd.DataFrame(y_values_mds)
PD_Y_values_mds.columns = ["LOWESS_x", "LOWESS_y"]

regression_mds = linregress(mds_df["genetic"], mds_df["euclidean"])
slope, intercept, r_value_mds, p_value, std_err = regression_mds

PD_Y_values_pca = pd.DataFrame(y_values_pca)
PD_Y_values_pca.columns = ["LOWESS_x", "LOWESS_y"]

regression_pca = linregress(pca_df["genetic"], pca_df["euclidean"])
slope, intercept, r_value_pca, p_value, std_err = regression_pca

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 6))
fig.tight_layout(pad=4.0)

ax1.plot(pca_df["genetic"], pca_df["euclidean"], "o", alpha=0.25)
ax1.plot(PD_Y_values_pca["LOWESS_x"], PD_Y_values_pca["LOWESS_y"], label="LOESS")

ax1.set_xlabel("Genetic distance")
ax1.set_ylabel("Euclidean distance (PCA)")
ax1.set_title("PCA Euclidean distance vs. genetic distance ($R^2=%.3f$)" % (r_value_pca ** 2))

ax2.plot(mds_df["genetic"], mds_df["euclidean"], "o", alpha=0.25)
ax2.plot(PD_Y_values_mds["LOWESS_x"], PD_Y_values_mds["LOWESS_y"], label="LOESS")

ax2.set_xlabel("Genetic distance")
ax2.set_ylabel("Euclidean distance (MDS)")
ax2.set_title("MDS Euclidean distance vs. genetic distance ($R^2=%.3f$)" % (r_value_mds ** 2))

ax3.plot(tsne_df["genetic"], tsne_df["euclidean"], "o", alpha=0.25)
ax3.plot(PD_Y_values_tsne["LOWESS_x"], PD_Y_values_tsne["LOWESS_y"], label="LOESS")

ax3.set_xlabel("Genetic distance")
ax3.set_ylabel("Euclidean distance (t-SNE)")
ax3.set_title("t-SNE Euclidean distance vs. genetic distance ($R^2=%.3f$)" % (r_value_tsne ** 2))

ax4.plot(umap_df["genetic"], umap_df["euclidean"], "o", alpha=0.25)
ax4.plot(PD_Y_values_umap["LOWESS_x"], PD_Y_values_umap["LOWESS_y"], label="LOESS")

ax4.set_xlabel("Genetic distance")
ax4.set_ylabel("Euclidean distance (UMAP)")
ax4.set_title("UMAP Euclidean distance vs. genetic distance ($R^2=%.3f$)" % (r_value_umap ** 2))


sns.despine()
plt.savefig("../docs/FullScatterplotZika.png")

## Within- and between-clade Euclidean distances for all embeddings

Use the complete embedding data frame to calculate pairwise Euclidean distances between samples and plot the results in a single figure.

In [ ]:
data_frames = [
    get_euclidean_data_frame(merged_df, "PCA1", "PCA2", "PCA"),
    get_euclidean_data_frame(merged_df, "MDS1", "MDS2", "MDS"),
    get_euclidean_data_frame(merged_df, "TSNE1", "TSNE2", "t-SNE"),
    get_euclidean_data_frame(merged_df, "UMAP1", "UMAP2", "UMAP"),
]

In [ ]:
merged_df.shape

Extract pairwise genetic (Hamming) distances corresponding to the records sampled above. This step assumes that the original merged data frame is indexed from zero to N for N total samples in the same order as the similarity matrix.

In [ ]:
genetic_distances = squareform(similarity_matrix)
len(genetic_distances)
len(data_frames[0])

In [ ]:
data_frames.append(pd.DataFrame({
    "distance": genetic_distances,
    "clade_status": data_frames[0]["clade_status"].values,
    "embedding": "genetic"
}))

In [ ]:
len(data_frames)

In [ ]:
euclidean_data_frame = pd.concat(data_frames)

In [ ]:
g = sns.FacetGrid(
    euclidean_data_frame,
    col="embedding",
    col_wrap=3,
    col_order=["genetic", "PCA", "MDS", "t-SNE", "UMAP"],
    sharey=False,
    height=4
)
g = g.map(sns.violinplot, "clade_status", "distance", order=["within", "between"])
g.set_axis_labels("Clade status", "Distance")

plt.savefig("../docs/FullViolinPlotZika.png")

In [ ]:
PCA_df = euclidean_data_frame[euclidean_data_frame.embedding == "PCA"]
MDS_df = euclidean_data_frame[euclidean_data_frame.embedding == "MDS"]
TSNE_df = euclidean_data_frame[euclidean_data_frame.embedding == "t-SNE"]
UMAP_df = euclidean_data_frame[euclidean_data_frame.embedding == "UMAP"]
genetic_df = euclidean_data_frame[euclidean_data_frame.embedding == "genetic"]

In [ ]:
genetic_df

In [ ]:
genetic_df['distance'].mean()

In [ ]:
median_genetic_within = genetic_df[genetic_df.clade_status == "within"].median()
median_genetic_between = genetic_df[genetic_df.clade_status == "between"].median()

In [ ]:
median_PCA_within = PCA_df[PCA_df.clade_status == "within"].median()
median_PCA_between = PCA_df[PCA_df.clade_status == "between"].median()

In [ ]:
median_MDS_within = MDS_df[MDS_df.clade_status == "within"].median()
median_MDS_between = MDS_df[MDS_df.clade_status == "between"].median()

In [ ]:
median_TSNE_within = TSNE_df[TSNE_df.clade_status == "within"].median()
median_TSNE_between = TSNE_df[TSNE_df.clade_status == "between"].median()

In [ ]:
median_UMAP_within = UMAP_df[UMAP_df.clade_status == "within"].median()
median_UMAP_between = UMAP_df[UMAP_df.clade_status == "between"].median()

In [ ]:
def ratioFunction(num1, num2):
    ratio12 = int(num1/num2)
    return ratio12

In [ ]:
genetic_ratio = ratioFunction(median_genetic_between,median_genetic_within)
print(genetic_ratio)

In [ ]:
PCA_ratio = ratioFunction(median_PCA_between,median_PCA_within)
print(PCA_ratio)

In [ ]:
MDS_ratio = ratioFunction(median_MDS_between,median_MDS_within)
print(MDS_ratio)

In [ ]:
TSNE_ratio = ratioFunction(median_TSNE_between,median_TSNE_within)
print(TSNE_ratio)

In [ ]:
UMAP_ratio = ratioFunction(median_UMAP_between,median_UMAP_within)
print(UMAP_ratio)